## Data Loader

In [1]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더


Bad key "text.kerning_factor" on line 4 in
/home/sungeun/anaconda3/envs/pytorch10/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
class Regressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(16, 8)
        self.fc2 = nn.Linear(8, 4)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        return x

In [3]:
from torchsummary import summary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Regressor().to(device)
summary(model, (1, 23343, 16))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1          [-1, 1, 23343, 8]             136
            Linear-2          [-1, 1, 23343, 4]              36
Total params: 172
Trainable params: 172
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 1.42
Forward/backward pass size (MB): 2.14
Params size (MB): 0.00
Estimated Total Size (MB): 3.56
----------------------------------------------------------------


## train

In [4]:
def train(log_interval, model, train_loader, optimizer, epoch):
    model.train()
    running_loss =0.0
    criterion = nn.CrossEntropyLoss()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} \tLoss: {:.6f}'.format(
                epoch, running_loss/log_interval))
            running_loss =0.0

In [5]:
epochs = 300
learning_rate = 0.001
batch_size = 256
log_interval =100

train_dataset = pd.read_csv('/home/sungeun/deep-high-resolution-net.pytorch-master/output/train_data.csv')
test_dataset = pd.read_csv('/home/sungeun/deep-high-resolution-net.pytorch-master/output/test_data.csv')

# train, test / direction
X_train = train_dataset.drop(['squat', 'direction'], axis=1)
y_train = train_dataset['squat']
X_val = test_dataset.drop(['squat', 'direction'], axis=1)
y_val = test_dataset['squat']
train_direction = train_dataset['direction']
test_direction = test_dataset['direction']

# dataframe -> numpy -> torch
X_train = torch.from_numpy(X_train.values).float()
y_train = torch.from_numpy(y_train.values).long()
X_val = torch.from_numpy(X_val.values).float()
y_val = torch.from_numpy(y_val.values).long()

# train dataloader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [6]:
model = Regressor().to(device)

optimizer = optim.Adam(model.parameters(),lr=learning_rate)

# train
for epoch in range(1, epochs + 1):
    train(log_interval, model, train_loader, optimizer, epoch)

Train Epoch: 1 	Loss: 0.014853
Train Epoch: 2 	Loss: 0.013433
Train Epoch: 3 	Loss: 0.011443
Train Epoch: 4 	Loss: 0.010801
Train Epoch: 5 	Loss: 0.010180
Train Epoch: 6 	Loss: 0.008922
Train Epoch: 7 	Loss: 0.008976
Train Epoch: 8 	Loss: 0.008350
Train Epoch: 9 	Loss: 0.008333
Train Epoch: 10 	Loss: 0.008652
Train Epoch: 11 	Loss: 0.008518
Train Epoch: 12 	Loss: 0.007763
Train Epoch: 13 	Loss: 0.008666
Train Epoch: 14 	Loss: 0.007691
Train Epoch: 15 	Loss: 0.007307
Train Epoch: 16 	Loss: 0.007136
Train Epoch: 17 	Loss: 0.006849
Train Epoch: 18 	Loss: 0.006561
Train Epoch: 19 	Loss: 0.006265
Train Epoch: 20 	Loss: 0.006519
Train Epoch: 21 	Loss: 0.005726
Train Epoch: 22 	Loss: 0.006844
Train Epoch: 23 	Loss: 0.005450
Train Epoch: 24 	Loss: 0.006218
Train Epoch: 25 	Loss: 0.005382
Train Epoch: 26 	Loss: 0.005652
Train Epoch: 27 	Loss: 0.005619
Train Epoch: 28 	Loss: 0.005508
Train Epoch: 29 	Loss: 0.005559
Train Epoch: 30 	Loss: 0.005029
Train Epoch: 31 	Loss: 0.005008
Train Epoch: 32 	

Train Epoch: 253 	Loss: 0.001498
Train Epoch: 254 	Loss: 0.001893
Train Epoch: 255 	Loss: 0.001967
Train Epoch: 256 	Loss: 0.001470
Train Epoch: 257 	Loss: 0.002153
Train Epoch: 258 	Loss: 0.002411
Train Epoch: 259 	Loss: 0.002363
Train Epoch: 260 	Loss: 0.001927
Train Epoch: 261 	Loss: 0.002259
Train Epoch: 262 	Loss: 0.001911
Train Epoch: 263 	Loss: 0.002537
Train Epoch: 264 	Loss: 0.001705
Train Epoch: 265 	Loss: 0.001626
Train Epoch: 266 	Loss: 0.001908
Train Epoch: 267 	Loss: 0.001799
Train Epoch: 268 	Loss: 0.001751
Train Epoch: 269 	Loss: 0.001655
Train Epoch: 270 	Loss: 0.001692
Train Epoch: 271 	Loss: 0.001777
Train Epoch: 272 	Loss: 0.002127
Train Epoch: 273 	Loss: 0.001492
Train Epoch: 274 	Loss: 0.002116
Train Epoch: 275 	Loss: 0.001359
Train Epoch: 276 	Loss: 0.002213
Train Epoch: 277 	Loss: 0.002017
Train Epoch: 278 	Loss: 0.001671
Train Epoch: 279 	Loss: 0.001821
Train Epoch: 280 	Loss: 0.002522
Train Epoch: 281 	Loss: 0.001734
Train Epoch: 282 	Loss: 0.001981
Train Epoc

In [7]:
# test
test_losses = []
test_loss = 0
correct = 0

criterion =  nn.CrossEntropyLoss(reduction='sum')

with torch.no_grad():
    model.eval()
    X_val, y_val = X_val.to(device), y_val.to(device)
    predictions = model(X_val)
    test_loss += criterion(predictions, y_val)
    pred = predictions.argmax(dim=1, keepdim=True) # get the index of the max log-probability
    correct += pred.eq(y_val.view_as(pred)).sum().item()

test_losses.append(test_loss)

test_loss /= len(X_val)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct,
                                                                             len(X_val),
                                                                             100. * correct / len(X_val)))


Test set: Average loss: 0.4653, Accuracy: 15148/17652 (86%)



In [8]:
# save
savePath = "/home/sungeun/deep-high-resolution-net.pytorch-master/demo/submission_1024.pth"
torch.save(model.state_dict(), savePath)

## Precision / Recall

In [9]:
confusionMatrix = np.zeros((4,4))
for i, y in enumerate(y_val):
    confusionMatrix[y, pred[i]] += 1
print(confusionMatrix)

[[7.844e+03 6.570e+02 5.200e+01 0.000e+00]
 [3.480e+02 2.682e+03 4.540e+02 0.000e+00]
 [9.000e+00 2.850e+02 2.696e+03 1.410e+02]
 [0.000e+00 3.000e+00 5.550e+02 1.926e+03]]


In [10]:
#Precision = avg(TP/TP+FP)
#Recall = avg(TP/TP+FN)

precisions = []
recalls = []
for i in range(4):
    TP = confusionMatrix[i, i]
    FP = 0
    FN = 0
    for j in range(4):
        if(i!=j):
            FP += confusionMatrix[j, i]
            FN += confusionMatrix[i, j]
            pre = TP / (TP+FP)
            re = TP / (TP+FN)
    precisions.append(pre)
    recalls.append(re)

avgPre = sum(precisions, 0.0)/len(precisions)
avgRe = sum(recalls, 0.0)/len(recalls)
F1_Score = 2*(avgPre*avgRe)/(avgPre+avgRe)

In [14]:
for i in range(4):
    print('Precision : ', precisions[i])
    print('Recall : ', recalls[i])
    print('\n')
print('------------------------------------')
print('AvgPre : ', avgPre)
print('AvgRe : ', avgRe)
print('------------------------------------')
print('F1 score : ', F1_Score)

Precision :  0.9564687233264236
Recall :  0.9171051093183679


Precision :  0.739454094292804
Recall :  0.769804822043628


Precision :  0.7175938248602608
Recall :  0.861066751836474


Precision :  0.9317851959361393
Recall :  0.7753623188405797


------------------------------------
AvgPre :  0.8363254596039069
AvgRe :  0.8308347505097624
------------------------------------
F1 score :  0.8335710633684073
